<a href="https://colab.research.google.com/github/nicoussj/MachineLearning/blob/main/SoportVectorMachine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Imports

In [ ]:
# Importación de librerías necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm
import warnings

# Configuraciones para gráficos
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0)

# Ignorar advertencias
warnings.filterwarnings('ignore')

##Dataset

In [ ]:
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
    name=fn
df = pd.read_csv(name, sep=";", encoding='latin1', header=0)

df.columns = df.columns.str.replace(';', '')

df.head(10)

df = df.dropna()
df['RoundStartingEquipmentValue'] = df['RoundStartingEquipmentValue'].fillna(df['RoundStartingEquipmentValue'].median())
df = pd.get_dummies(df, columns=['Map'], drop_first=True)
#Cambio de datos: Cambiar de False4 en RoundWinner a False, agregar False a MatchWinner, ya que en contexto aplica
df.loc[29,'MatchWinner'] = 'False'
df.loc[29,'RoundWinner'] = 'False'
#Se eliminará columna Unnamed: 0, ya que solo contiene las ID de las filas
#Se eliminará AbnormalMatch, ya que contiene solamente valores False
df.drop(columns=['Unnamed: 0'], inplace=True)
df.drop(columns=['AbnormalMatch'], inplace=True)

Saving Anexo ET_demo_round_traces_2022.csv to Anexo ET_demo_round_traces_2022.csv


##SoportVectorMachine

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
from sklearn.preprocessing import StandardScaler

# -------------------------------
# 1. Preparar los datos
# -------------------------------

# Seleccionar columnas
features_reg = ['MatchHeadshots', 'RoundStartingEquipmentValue', 'TeamStartingEquipmentValue', 'PrimaryAssaultRifle']
target_reg = 'MatchKills'

df_model_reg = df[features_reg + [target_reg]].copy()
df_model_reg = df_model_reg.dropna()

# Asegurar que 'PrimaryAssaultRifle' sea numérico
df_model_reg['PrimaryAssaultRifle'] = pd.to_numeric(df_model_reg['PrimaryAssaultRifle'], errors='coerce')
df_model_reg = df_model_reg.dropna()

# Dividir en X e y
X = df_model_reg[features_reg]
y = df_model_reg[target_reg]

# Escalar los datos (muy importante para SVR)
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1)).ravel()

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.3, random_state=42)

# -------------------------------
# 2. Entrenar el modelo SVR
# -------------------------------

# Crear modelo SVR (puedes probar con diferentes kernels: 'linear', 'poly', 'rbf')
svr_model = SVR(kernel='rbf', C=100, epsilon=0.1)
svr_model.fit(X_train, y_train)

# -------------------------------
# 3. Predecir y desescalar los resultados
# -------------------------------

# Predecir
y_pred_scaled = svr_model.predict(X_test)

# Desescalar las predicciones y valores reales
y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).ravel()
y_test_real = scaler_y.inverse_transform(y_test.reshape(-1, 1)).ravel()

# -------------------------------
# 4. Evaluar el modelo
# -------------------------------

mae_svr = mean_absolute_error(y_test_real, y_pred)
rmse_svr = np.sqrt(mean_squared_error(y_test_real, y_pred))
r2_svr = r2_score(y_test_real, y_pred)

print(f"📊 Evaluación del modelo SVR:")
print(f"MAE  (Error Absoluto Medio):     {mae_svr:.2f}")
print(f"RMSE (Raíz del Error Cuadrático): {rmse_svr:.2f}")
print(f"R²   (Coef. de Determinación):    {r2_svr:.2f}")

📊 Evaluación del modelo SVR:
MAE  (Error Absoluto Medio):     2.44
RMSE (Raíz del Error Cuadrático): 3.36
R²   (Coef. de Determinación):    0.71
